In [3]:
import cv2
import numpy as np

In [30]:
def generate_transformed_image(src, transform_matrix):
    rows, cols = src.shape
    dst = cv2.warpAffine(src, transform_matrix, (cols, rows))
    return dst

def add_imperfections_to_points(points, variance=0.05):
    """
    Add slight random variations to the points to simulate imperfections in feature matching.
    
    Parameters:
        points (np.array): The original points.
        variance (float): The variance of the Gaussian noise to be added.
        
    Returns:
        np.array: The points with added imperfections.
    """
    noise = np.random.normal(0, variance, points.shape)
    return points + noise

In [33]:
# Initialize a synthetic image
src = np.zeros((100, 100), dtype=np.uint8)
cv2.rectangle(src, (30, 30), (70, 70), 255, thickness=-1)

# Define a small rotation and translation transformation
angle = 0.0  # 1 degree
scale = np.random.rand(3000) * 0.08 + 0.96
scale = scale / scale.mean()
center = (src.shape[1] / 2, src.shape[0] / 2)


In [34]:
from math import prod
prod(scale)

0.45239678147258433

In [72]:
cumulative_transform = np.eye(2, 3, dtype=np.float32)  # Initialize cumulative transformation matrix
for i in range(3000):
    # Generate transformed image
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale[i])
    dst = generate_transformed_image(src, rotation_matrix)
    
    # Simulate perfect feature matching
    src_points = np.array([[30, 30], [30, 70], [70, 70], [70, 30]], dtype=np.float32)
    perfect_dst_points = np.float32([cv2.transform(np.array([[p]], dtype=np.float32), rotation_matrix)[0][0] for p in src_points])
    imperfect_dst_points = add_imperfections_to_points(perfect_dst_points, variance=0.05)  # Add imperfections


    # Estimate transformation from src to dst
    estimated_transform, inliers = cv2.estimateAffinePartial2D(src_points, imperfect_dst_points)
    
    if estimated_transform is not None:
        # Correctly extend the 2x3 affine transformation matrix to a 3x3 matrix for proper matrix multiplication
        extended_estimated_transform = np.vstack([estimated_transform, [0, 0, 1]])
        
        # Update the cumulative transformation matrix
        cumulative_transform_extended = np.vstack([cumulative_transform, [0, 0, 1]])
        cumulative_transform = np.dot(extended_estimated_transform, cumulative_transform_extended)
        cumulative_transform = cumulative_transform[:2, :]
    
    src = dst.copy()

# Print the cumulative transformation matrix
print("Cumulative Transformation Matrix:\n", cumulative_transform)

Cumulative Transformation Matrix:
 [[ 3.99621337e-01 -7.93277779e-04  2.97830086e+01]
 [ 7.93277779e-04  3.99621337e-01  2.92732501e+01]]


In [15]:
estimated_transform.shape

(2, 3)